# PDF -> Images

In [ ]:
from pdf2image import convert_from_path
import os

in_path = "input_pdf/Subiect_MG.pdf"
out_path = "pdf2images"
os.makedirs(out_path, exist_ok=True)

images = convert_from_path(in_path, dpi=300)

for i, image in enumerate(images, start=1):
    filename = f"pagina_{i}.png"
    image.save(os.path.join(out_path, filename), "PNG")

print("Imaginile au fost salvate în:", out_path)


Imaginile au fost salvate în: pdf2images


# Detectare pozitii grile

In [ ]:
import cv2
import numpy as np

def detect_questions(img_path):
    img = cv2.imread(img_path)
    grayscale_img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    _, prag = cv2.threshold(grayscale_img, 150, 255, cv2.THRESH_BINARY_INV)

    contururi, _ = cv2.findContours(prag, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    question_boxes = []
    for cnt in contururi:
        x, y, w, h = cv2.boundingRect(cnt)
        if h > 100 and w > 200:
            question_boxes.append((x, y, x + w, y + h))
    return sorted(question_boxes, key=lambda x: x[1])

img -> img_grayscale (scade din complexitate inainte de a face operatii) -> binarizare -> cautare contururi -> returnare coordonate dreptunghiuri cu grile

question_boxes va contine coordonatele dreptunghiurilor intrebarilor

In [27]:
from PIL import Image

questions_cropped = []
for i in range(len(images)):
    img_path = os.path.join(out_path, f"pagina_{i+1}.png")
    boxes = detect_questions(img_path)
    page_img = Image.open(img_path)
    for box in boxes:
        questions_cropped.append(page_img.crop(box))

cropped_out_path = "intrebari_decupate"
os.makedirs(cropped_out_path, exist_ok=True)

for i, cropped in enumerate(questions_cropped, start=1):
    cropped.save(os.path.join(cropped_out_path, f"intrebare_{i}.png"))


# Alegem random 35 de grile si le punem intr-un PDF 

In [30]:
import random

os.makedirs("output_pdf", exist_ok=True)

selected = random.sample(questions_cropped, 2)

selected[0].save("output_pdf/output.pdf", save_all=True, append_images=selected[1:], resolution=300)
